<a href="https://colab.research.google.com/github/ZoneSixGames/ML-Prompt-Scripts/blob/main/MadLib_Prompt_Generator_by_remi_durant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mad-libbed Prompt Generator
by [@remi_durant](https://twitter.com/remi_durant)

Module to easily make random mad-lib style prompts in any AI art notebook.

![visitors](https://visitor-badge.glitch.me/badge?page_id=remi_madlibbing)

In [ ]:
#@title Madlibbed Prompt Generator Module (Simple)
#@markdown Pass in a madlib style prompt and a lookup dictionary to get a random prompt
#@markdown 
#@markdown -----------------
#@markdown Function Options
#@markdown - `avoid_repeats` : When picking words to fill the mad lib slots, should repeats be avoided?
#@markdown - `fix_indefinite_articles` : When filling mad libs, should a A/An prior to the slot be fixed up?
#@markdown 
#@markdown <font size="2">*Madlib Prompt Module by [@remi_durant](https://twitter.com/remi_durant)*</font>

import re
import random

def madlib_prompt(base, lookup = {},
        options = { 
            "avoid_repeats" : True, 
            "fix_indefinite_articles" : True 
        }):
    lookup_cardstack = {}

    def on_madlib_sub( match ):
        opt = match.group(2).split('|')

        key = opt[0]

        # do a lookup if there's only one option in the brackets
        if len(opt) == 1 and lookup.get(key):

            # discard used words to avoid repeats, unless no words remain
            if options["avoid_repeats"]:

                if len(lookup_cardstack.get(key) or []) == 0:
                    lookup_cardstack[key] = copy.copy( lookup[key] )

                g2 = random.choice( lookup_cardstack[key] )
                lookup_cardstack[key].remove(g2)

               
            else:
                g2 = random.choice( lookup.get(key) )

        # choose one of the options to fill this space
        else:
            g2 = random.choice( opt )

        # if the previous word is 'A' or 'An', figure out if the 'n' is needed
        g1 = match.group(1)
        if g1 is not None:
            if options["fix_indefinite_articles"]:
                if g2[0].lower() in list("aeiou"):
                    g1 = g1[0] + 'n'
                else:
                    g1 = g1[0]
            g2 = g1 + ' ' + g2
        
        return g2
    
    # find madlib spots, and replace
    return re.sub(r"(\b[Aa]n? )?{([^{}]*)}", on_madlib_sub, base )


When writing your base prompt, there are two mad-lib style features you can use:

### In-Place Alternates
 You can put alternate options within brackets split by pipes and it will pick one at random. 
 

In [ ]:
madlib_prompt( '{red|blue} car' )

'blue car'

### Lookup Alternates
You can also specify a key to look for in a passed in lookup table. 

In [ ]:
madlib_prompt( '{color} car', {
    'color': ['red','blue','white','orange','green','purple'] 
} )

'purple car'

This way you can create a lot more diversity in the prompt without the base prompt becoming unmaneagable.

### Combining
You can use both techniques in the same prompt to make much more complicated random prompts.

In [ ]:
ml_prompt = "A {something} Garden by {Greg Rutkowski|James Gurney}" 
ml_lookup =  {
     'something': ['Enchanted','Forbidden','Exotic','Vivid'] 
}

for i in range(5):
    print( madlib_prompt( ml_prompt, ml_lookup ) )

A Forbidden Garden by James Gurney
An Exotic Garden by Greg Rutkowski
A Vivid Garden by Greg Rutkowski
An Exotic Garden by James Gurney
An Enchanted Garden by James Gurney


You can see in this example how it also automatically converts A/An to the appropriate one for the prompt, based on if the mad lib word starts with a vowel. This isn't technically correct, but it covers most of the normal words.

## Advanced Mode

In [ ]:
#@title Madlibbed Prompt Generator Module (Advanced)
#@markdown Pass in an array of templates, and a lookup dictionary.
#@markdown 
#@markdown -----------------
#@markdown Function Options
#@markdown - `avoid_repeats` : When picking words to fill the mad lib slots, should repeats be avoided?
#@markdown - `fix_indefinite_articles` : When filling mad libs, should a A/An prior to the slot be fixed up?
#@markdown 
#@markdown -----------------
#@markdown Template parameters
#@markdown - `prompt` : the mad lib style template to process
#@markdown - `lookup` : any prompt specific madlib lists, or overrides of the global ones
#@markdown - `weight` : how much should this template be favored when picking a template
#@markdown 
#@markdown <font size="2">*Madlib Prompt Module by [@remi_durant](https://twitter.com/remi_durant)*</font>


import re
import random
import copy

def madlib_template( templates, global_lookup = {}, 
        options = { 
            "avoid_repeats" : True, 
            "fix_indefinite_articles" : True 
        }):

    # get prompt template weights and pick one
    weights = [ p.get('weight') or 1 for p in templates]
    t = random.choices( templates, weights=weights, k=1 )[0]
    
    lookup = { **global_lookup, **(t.get('lookup') or {}) }
    lookup_cardstack = {}

    def on_madlib_sub( match ):
        opt = match.group(2).split('|')

        key = opt[0]

        # do a lookup if there's only one option in the brackets
        if len(opt) == 1 and lookup.get(key):

            # discard used words to avoid repeats, unless no words remain
            if options["avoid_repeats"]:

                if len(lookup_cardstack.get(key) or []) == 0:
                    lookup_cardstack[key] = copy.copy( lookup[key] )

                g2 = random.choice( lookup_cardstack[key] )
                lookup_cardstack[key].remove(g2)

            else:
                g2 = random.choice( lookup.get(key) )
          
        # or just pick one of the given options  
        else:
            g2 = random.choice( opt )

        g1 = match.group(1)
        if g1 is not None:

            # if the previous word is 'A' or 'An', figure out if the 'n' is needed
            if options["fix_indefinite_articles"]:
                    if g2[0].lower() in list("aeiou"):
                        g1 = g1[0] + 'n'
                    else:
                        g1 = g1[0]
            g2 = g1 + ' ' + g2
            
        return g2
    
    # find madlib spots, and replace
    return re.sub(r"(\b[Aa]n? )?{([^{}]*)}", on_madlib_sub, t['prompt'] )


With the advanced version of this module you can specify multiple prompt templates and give them different weightings and lookup table overrides per base prompt.

In [ ]:
ml_templates = [ 
  { 
      "prompt" : "A {something}, {something} garden by {James Gurney|Greg Rutkowski}", 
      "lookup" : {}, 
      "weight":2 # increase the frequency of this prompt
  },
  {   
      "prompt" : "A beautiful {color} {sunrise|sunset} over the {something} {something} garden by {James Gurney|Greg Rutkowski}", 
      "lookup" : {
          "something": ["enchanted"], # overriding the base set
          "color" : ['blue','red','purple','pink'] # adding extra data for only this prompt
      } 
  } 
]
ml_lookup = {
    "something":['vivid','enchanted','forbidden','rotten']
}

for i in range(5):
    print( madlib_template( ml_templates, ml_lookup, options = { 
        "avoid_repeats" : True, 
        "fix_indefinite_articles" : True 
    } ) )


A vivid, forbidden garden by Greg Rutkowski
A vivid, rotten garden by James Gurney
A beautiful pink sunset over the enchanted enchanted garden by Greg Rutkowski
A beautiful blue sunrise over the enchanted enchanted garden by James Gurney
A vivid, forbidden garden by Greg Rutkowski


## Full Example using some larger Prompt Lists

In [ ]:
#@title Download some prompt lists
#@markdown These lists were pulled from [Neuralism Prompt Generator](https://colab.research.google.com/drive/1mrlY_mc-HdIxEHILY2BvA9u0mOjD2Ze5#scrollTo=GBRlz7DRueqx).
!git clone https://github.com/sanzelda/prompt_gen.git

In [ ]:
#@title Load prompt lists from text files into a lookup table
import os
prompt_lookup = [ f.split('.') for f in os.listdir("/content/prompt_gen/") ]
prompt_lookup = { f[0] : open(f"/content/prompt_gen/{f[0]}.txt").read().splitlines() for f in prompt_lookup if len(f) == 2 and f[1] == "txt" }
print( prompt_lookup.keys() )

dict_keys(['adjectives', 'suffixes', 'things', 'artists', 'shapes', 'locations', 'animals', 'colors'])


In [ ]:
#@title Make some prompts!
ml_templates = [ 
  { 
      "prompt" : "A {adjectives} {shapes} {locations} by {artists} and {artists}", 
      "lookup" : {}, 
      "weight":1 # increase the frequency of this prompt
  },
  
  { 
      "prompt" : "A {shapes} {animals} in a {adjective} {locations} by {artists} and {artists}", 
      "lookup" : {}, 
      "weight":1 # increase the frequency of this prompt
  }
]
ml_lookup = {
    **prompt_lookup,
    "something":['vivid','enchanted','forbidden','rotten']
}

for i in range(5):
    print( madlib_template( ml_templates, ml_lookup, options = { 
        "avoid_repeats" : True, 
        "fix_indefinite_articles" : True 
    } ) )


A technological rhombus universe by Jackson Pollock and Hieronymus Bosch
A luminus heights village by Egon Schiele and Mark Rothko
A feathers amberjack in an adjective ocean by H.P. Lovecraft and Dan Mumford
An octagons chick in an adjective ocean by Pierre-Auguste Renoir and Johannes Vermeer
A beads and yarn grouper in an adjective country by Paul Gauguin and Vincent Di Fate
